# Imports and Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU, ReLU

from keras.applications import EfficientNetB5

USE_GPU = True

2024-03-07 17:16:04.076060: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 17:16:04.217288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 17:16:04.217388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 17:16:04.225481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 17:16:04.278782: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.threading.set_inter_op_parallelism_threads(
    2
)

In [2]:
if USE_GPU:
    #tf.debugging.set_log_device_placement(True)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

2024-03-07 16:44:28.820320: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:28.849134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:28.849219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



2024-03-07 16:44:29.122801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.122901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.122910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-03-07 16:44:29.122937: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.122958: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

In [21]:
## Define path to data and parameters for loader
TRAIN_PATH = './data/training/'
TEST_PATH = './data/testing/'
LABELS_PATH = './data/training_labels.csv'

TEST_SIZE = 0.05
BATCH_SIZE = 16
#IMG_HEIGHT = 180
#IMG_WIDTH = 180

N_CLASSES = 5

## Encoding labels
LABEL_ENCODING = {
    'daisy': 0,
    'dandelion': 1,
    'rose': 2,
    'sunflower': 3,
    'tulip': 4
}

TARGET_SIZE = (456, 456)

Read in data and assign labels

In [30]:
tf.config.threading.set_inter_op_parallelism_threads(
    2
)

RuntimeError: Inter op parallelism cannot be modified after initialization.

In [17]:
## Get labels for train and test data
labels_df = pd.read_csv(LABELS_PATH)
labels_df['full_id'] = [os.path.join('/home/scottbrown/byu/stat486/projects/flower-image-classification/data/training/training', l) for l in labels_df.ID]

In [5]:
# labels_df['target_encoded'] = labels_df['target'].copy() #.map(LABEL_ENCODING)

# train_labels = list(zip(labels_df['ID'], labels_df['target_encoded']))
# test_labels = list(zip(labels_df['ID'], labels_df['target_encoded']))

# ## Sort (in place) according to the alphanumeric order of the image file paths.
# train_labels.sort(key=lambda x: x[0])
# test_labels.sort(key=lambda x: x[0])

## Change first item in each tuple to index
# # train_labels = [(i, l[1]) for i, l in enumerate(train_labels)]
# # test_labels = [(i, l[1]) for i, l in enumerate(test_labels)]

In [ ]:
print(os.listdir(TRAIN_PATH))
print(os.listdir(TEST_PATH))

In [22]:
## NOTE: `flow_from_dataframe` via ImageDataGenerator is deprecated
# In the future, organize directory structure for `flow_from_directory`
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=TEST_SIZE)
train_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    target_size=TARGET_SIZE,
    subset='training',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

test_gen = datagen.flow_from_dataframe(
    labels_df, 
    directory=TRAIN_PATH, 
    target_size=TARGET_SIZE,
    subset='validation',
    x_col='full_id', 
    y_col='target', 
    class_mode='categorical', 
    batch_size=BATCH_SIZE)

## Read in test data to predict on
new_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
new_gen = new_datagen.flow_from_directory(
    TEST_PATH, 
    target_size=TARGET_SIZE, 
    class_mode=None, 
    shuffle=False, 
    batch_size=1)

class_names = list(train_gen.class_indices.keys())

Found 3282 validated image filenames belonging to 5 classes.


Found 172 validated image filenames belonging to 5 classes.
Found 863 images belonging to 1 classes.


In [7]:
# train = tf.keras.utils.image_dataset_from_directory(
#   TRAIN_PATH,
#   labels=train_labels,
#   #label_mode = 'categorical',
#   validation_split=TEST_SIZE,
#   subset='training',
#   seed=9, # Joe Burrow
#   #image_size=(img_height, img_width),
#   batch_size=BATCH_SIZE)

# test = tf.keras.utils.image_dataset_from_directory(
#   TRAIN_PATH,
#   labels=test_labels,
#   #label_mode = 'categorical',
#   validation_split=TEST_SIZE,
#   subset='validation',
#   seed=9, # Joe Burrow
#   #image_size=(img_height, img_width),
#   batch_size=BATCH_SIZE)

# Preview images

In [8]:
# ## Recreate the above code for train_gen generator object
# class_names = list(train_gen.class_indices.keys())
# for images, labels in train_gen:
#   plt.figure(figsize=(10, 10))
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     lab = class_names[labels[i].argmax()]
#     plt.imshow(images[i])
#     plt.title(lab)
#     plt.axis("off")
#   break

# Create model

In [9]:
base_model = EfficientNetB5(weights='imagenet', include_top=False, drop_connect_rate=0.4, input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))
UNFREEZE_N = 2

# Freeze all but N layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Optionally, unfreeze the top N layers
for layer in base_model.layers[-UNFREEZE_N:]:
    layer.trainable = True

#x = base_model.output

2024-03-07 16:44:29.274610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.274726: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.274742: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.275091: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 16:44:29.275150: I external/local_xla/xla/stream_executor

In [23]:
cnn = tf.keras.Sequential([
    ## Use base model for transfer learning
    base_model,

    ## 3 Convolutional => MaxPooling layers
    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    # tf.keras.layers.Conv2D(32, (2, 2)),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.ReLU(),
    # #tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.4),

    # ## 1 Fully connected layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),#LeakyReLU(alpha=0.01)),
    
    # ## Output layer
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])

# cnn = tf.keras.Sequential([
#     ## Use base model for transfer learning
#     base_model,

#     ## 3 Convolutional => MaxPooling layers
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.4),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(32, activation='relu'),
    
#     tf.keras.layers.Dense(N_CLASSES, activation='softmax')
# ])


In [11]:
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
# for layer in base_model.layers:
#     layer.trainable = False

# x = base_model.output
# x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# predictions = Dense(4, activation='softmax')(x)

# model_transfer = Model(inputs=base_model.input, outputs=predictions)

# model_transfer.compile(optimizer='adam',
#                        loss='categorical_crossentropy',
#                        metrics=['accuracy'])

# model_transfer.fit(train_generator, validation_data=validation_generator, epochs=10)


In [25]:
cnn.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Fit model

In [26]:
checkpoint_filepath = '/tmp/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

cnn.fit( # TODO: checkpoints
  train_gen,
  validation_data=test_gen,
  callbacks=[model_checkpoint_callback],
  epochs=2#25
)

Epoch 1/2


2024-03-07 17:09:35.738641: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb5/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


206/206 [==============================] - 124s 552ms/step - loss: 1.8789 - accuracy: 0.2102 - val_loss: 1.7601 - val_accuracy: 0.2267
Epoch 2/2
206/206 [==============================] - 108s 525ms/step - loss: 1.6829 - accuracy: 0.2249 - val_loss: 1.6673 - val_accuracy: 0.1919


In [20]:
test_eval = cnn.evaluate(test_gen)#X_test, y_test, verbose=0)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

54/54 [==============================] - 26s 480ms/step - loss: 1.6034 - accuracy: 0.2375
Test loss: 1.6033602952957153
Test accuracy: 0.23754344880580902


# Predict new data

In [15]:
y_pred = cnn.predict(new_gen)

694/863 [=======================>......] - ETA: 8s

KeyboardInterrupt: 

In [35]:
pred_files = [f.split('/')[-1] for f in new_gen.filenames]
pred_labels = [class_names[i] for i in y_pred.argmax(axis=1)]
submission_df = pd.DataFrame({'ID': pred_files, 'Prediction': pred_labels})

In [37]:
## Write to CSV
submission_df.to_csv('submission.csv', index=False)